<a href="https://colab.research.google.com/github/Gunehee/Kaggle_StoreSales_project/blob/main/Store_Sales_Project_main.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Data
First of all I propose to explore the dataset.

Our goal is to predict future sales of stores located in Ecuador, for the dates August 16, 2017 to August 31, 2017 (16 days).

In our dataset, there are 54 stores for 33 product families.

We need to predict the sales for each of these product families from each store. So 33 * 54 * 16 = 28,512 values to predict.

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
#import the necessary libraries
import numpy as np
import pandas as pd

In [ ]:
#import the data
path = '/content/drive/MyDrive/DS_Army/Store Sales/Store_Sales_Data'

data_oil = pd.read_csv(path+'/oil.csv')
data_holidays = pd.read_csv(path+'/holidays_events.csv')
data_stores = pd.read_csv(path+'/stores.csv')
data_train = pd.read_csv(path+'/train.csv')
data_transactions = pd.read_csv(path+'/transactions.csv')
data_test = pd.read_csv(path+'/test.csv')
data_sample_submission = pd.read_csv(path+'/sample_submission.csv')

#print out general info abput the datasets
names = ['data_oil', 'data_holidays', 'data_stores', 'data_train', 'data_transactions', 'data_test']
for name, data in zip(names, [data_oil, data_holidays, data_stores, data_train, data_transactions, data_test]):
    print('shape of', name, data.shape)
    #print('columns in', name , ':', data.columns )
    print ('column types in', name ,':')
    print(data.dtypes)
    missing_values = data_train[data_train.isnull().any(axis=1)]
    if missing_values.empty:
        print('There are no missing values')
    else:
        print(missing_values)
    display(data.head())
    print('\n')

shape of data_oil (1218, 2)
column types in data_oil :
date           object
dcoilwtico    float64
dtype: object
There are no missing values


,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20




shape of data_holidays (350, 6)
column types in data_holidays :
date           object
type           object
locale         object
locale_name    object
description    object
transferred      bool
dtype: object
There are no missing values


,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False




shape of data_stores (54, 5)
column types in data_stores :
store_nbr     int64
city         object
state        object
type         object
cluster       int64
dtype: object
There are no missing values


...

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0




shape of data_transactions (83488, 3)
column types in data_transactions :
date            object
store_nbr        int64
transactions     int64
dtype: object
There are no missing values


,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922




shape of data_test (28512, 5)
column types in data_test :
id              int64
date           object
store_nbr       int64
family         object
onpromotion     int64
dtype: object
There are no missing values


,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


## train.csv
First of all the main file: train.csv. It contains some features and the label to predict sales, the number of sales per day:

In [ ]:
import pandas as pd
display(data_train.head())

,id,date,store_nbr,family,sales,onpromotion
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0
1,1,2013-01-01,1,BABY CARE,0.0,0
2,2,2013-01-01,1,BEAUTY,0.0,0
3,3,2013-01-01,1,BEVERAGES,0.0,0
4,4,2013-01-01,1,BOOKS,0.0,0


### Here are the columns of the DataFrame:

* id – the index of the row
* date – the current date
* store_nbr – the store
* family – the product family
* sales – number of sales in this family
* onpromotion – the number of products on promotion in this family

## holidays_events.csv
The holidays_events.csv groups the national holidays. This information is independent of the store but can have an impact on sales.

For example, on a holiday, there might be more people in the city and therefore more customers in the stores. Or conversely, more people might go on vacation and therefore there would be fewer customers in the stores.

In [ ]:
display(data_holidays.head())

,date,type,locale,locale_name,description,transferred
0,2012-03-02,Holiday,Local,Manta,Fundacion de Manta,False
1,2012-04-01,Holiday,Regional,Cotopaxi,Provincializacion de Cotopaxi,False
2,2012-04-12,Holiday,Local,Cuenca,Fundacion de Cuenca,False
3,2012-04-14,Holiday,Local,Libertad,Cantonizacion de Libertad,False
4,2012-04-21,Holiday,Local,Riobamba,Cantonizacion de Riobamba,False


### Here are the columns in the DataFrame:

* date – the date of the holiday
* type – the type of holiday (Holiday, Event, Transfer (see transferred column), Additional, Bridge, Work Day)
* locale – the scope of the event (Local, Regional, National)
* locale_name – the city where the event takes place
* description – name of the event
* transferred – whether the event has been transferred (moved to another day) or not

## oil.csv
Then a CSV file gathers the daily oil price from January 01, 2013 to August 31, 2017:

In [ ]:
display(data_oil.head())

,date,dcoilwtico
0,2013-01-01,NaN
1,2013-01-02,93.14
2,2013-01-03,92.97
3,2013-01-04,93.12
4,2013-01-07,93.20


## store.csv
The store.csv file gathers information about the stores. There is one store per line so 54 lines:

In [ ]:
display(data_stores.head())

,store_nbr,city,state,type,cluster
0,1,Quito,Pichincha,D,13
1,2,Quito,Pichincha,D,13
2,3,Quito,Pichincha,D,8
3,4,Quito,Pichincha,D,9
4,5,Santo Domingo,Santo Domingo de los Tsachilas,D,4


### DataFrame columns:

* store_nbr – the store
* city – the city where the store is located
* state – the state where the store is located
* type – the type of the store
* cluster – the number of similar stores in the vicini

## transactions.csv
The transactions.csv file groups the daily transactions by stores:

In [ ]:
# Transaction is a receipt created after a customer's purchase
display(data_transactions.head())

,date,store_nbr,transactions
0,2013-01-01,25,770
1,2013-01-02,1,2111
2,2013-01-02,2,2358
3,2013-01-02,3,3487
4,2013-01-02,4,1922


## test.csv
Finally, we have the test.csv that will allow us to predict the sale column. The file starts on August 16, 2017 and ends on August 31, 2017. We also have the sample_submission.csv to fill in with the number of sales per day and per family:

In [ ]:
display(data_test.head())
display(data_sample_submission.head())

,id,date,store_nbr,family,onpromotion
0,3000888,2017-08-16,1,AUTOMOTIVE,0
1,3000889,2017-08-16,1,BABY CARE,0
2,3000890,2017-08-16,1,BEAUTY,2
3,3000891,2017-08-16,1,BEVERAGES,20
4,3000892,2017-08-16,1,BOOKS,0


,id,sales
0,3000888,0.0
1,3000889,0.0
2,3000890,0.0
3,3000891,0.0
4,3000892,0.0


### The test.csv contains 5 columns:

* id – the index of the row (which will be used to fill in the sample_submission.csv file) date – the current date
* store_nbr – the store
* family – the product family
* sales – the number of sales in this family
* onpromotion – the number of products on promotion in this family

## Preprocessing
To start the preprocessing, let's group the name of each product family and the number of each store:

In [ ]:
family_list = data_train['family'].unique()
store_list = data_stores['store_nbr'].unique()
display(family_list)
display(store_list)

array(['AUTOMOTIVE', 'BABY CARE', 'BEAUTY', 'BEVERAGES', 'BOOKS',
       'BREAD/BAKERY', 'CELEBRATION', 'CLEANING', 'DAIRY', 'DELI', 'EGGS',
       'FROZEN FOODS', 'GROCERY I', 'GROCERY II', 'HARDWARE',
       'HOME AND KITCHEN I', 'HOME AND KITCHEN II', 'HOME APPLIANCES',
       'HOME CARE', 'LADIESWEAR', 'LAWN AND GARDEN', 'LINGERIE',
       'LIQUOR,WINE,BEER', 'MAGAZINES', 'MEATS', 'PERSONAL CARE',
       'PET SUPPLIES', 'PLAYERS AND ELECTRONICS', 'POULTRY',
       'PREPARED FOODS', 'PRODUCE', 'SCHOOL AND OFFICE SUPPLIES',
       'SEAFOOD'], dtype=object)

array([ 1,  2,  3,  4,  5,  6,  7,  8,  9, 10, 11, 12, 13, 14, 15, 16, 17,
       18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34,
       35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51,
       52, 53, 54])

Next, we assemble the df_train and df_stores datasets. By grouping this data in a single dataset, it will allow us to access the information more easily. In addition to that, we sort the sales of the DataFrame by date, by family and by stores:

In [ ]:
train_merged = pd.merge(data_train, data_stores, on ='store_nbr')
train_merged = train_merged.sort_values(["store_nbr","family","date"])
train_merged = train_merged.astype({"store_nbr":'str', "family":'str', "city":'str',
                          "state":'str', "type":'str', "cluster":'str'})

display(train_merged.head())

,id,date,store_nbr,family,sales,onpromotion,city,state,type,cluster
0,0,2013-01-01,1,AUTOMOTIVE,0.0,0,Quito,Pichincha,D,13
1782,1782,2013-01-02,1,AUTOMOTIVE,2.0,0,Quito,Pichincha,D,13
3564,3564,2013-01-03,1,AUTOMOTIVE,3.0,0,Quito,Pichincha,D,13
5346,5346,2013-01-04,1,AUTOMOTIVE,3.0,0,Quito,Pichincha,D,13
7128,7128,2013-01-05,1,AUTOMOTIVE,5.0,0,Quito,Pichincha,D,13


A time series being the number of sales made per day for a family of a store, this sorting will allow us to extract them more easily.

Same thing for the test DataFrame, we sort the sales by date, by family and by stores:

In [ ]:
df_test_dropped = data_test.drop(['onpromotion'], axis=1)
df_test_sorted = df_test_dropped.sort_values(by=['store_nbr','family'])

display(df_test_sorted.head())

,id,date,store_nbr,family
0,3000888,2017-08-16,1,AUTOMOTIVE
1782,3002670,2017-08-17,1,AUTOMOTIVE
3564,3004452,2017-08-18,1,AUTOMOTIVE
5346,3006234,2017-08-19,1,AUTOMOTIVE
7128,3008016,2017-08-20,1,AUTOMOTIVE


## Main Time Series

In [ ]:
!pip install darts==0.23.1
#&> /dev/null

### sales
**Extract the time series**

For each product family, we will gather all the time series concerning it.

So we will have 33 sub-datasets. These datasets will be contained in the family_TS_dict dictionary.

In the following lines of code, we extract the TimeSeries of the 54 stores for each family.

These TimeSeries will group the sales by family, the date of each sale, but also the dependent covariates (indicated with group_cols and static_cols) of these sales: store_nbr, family, city, state, type, cluster :

In [ ]:
import numpy as np
import darts
from darts import TimeSeries

family_TS_dict = {}

for family in family_list:
  df_family = train_merged.loc[train_merged['family'] == family]

  list_of_TS_family = TimeSeries.from_group_dataframe(
                                df_family,
                                time_col="date",
                                group_cols=["store_nbr","family"],
                                static_cols=["city","state","type","cluster"],
                                value_cols="sales",
                                fill_missing_dates=True,
                                freq='D')
  for ts in list_of_TS_family:
            ts = ts.astype(np.float32)

  list_of_TS_family = sorted(list_of_TS_family, key=lambda ts: int(ts.static_covariates_values()[0,0]))
  family_TS_dict[family] = list_of_TS_family

You can also see that we indicate fill_missing_dates=True because in the dataset, the sales of each December 25th are missing.

We also indicate freq='D', to indicate that the interval for the values of the time series is in days (D for day).

Finally, we indicate that the values of the TimeSeries must be interpreted in float32 and that the time series must be sorted by stores.

We can display the first time series of the first family:

In [ ]:
display(family_TS_dict['AUTOMOTIVE'][0])

<TimeSeries (DataArray) (date: 1688, component: 1, sample: 1)> Size: 14kB
array([[[0.]],

       [[2.]],

       [[3.]],

       ...,

       [[1.]],

       [[1.]],

       [[4.]]])
Coordinates:
  * date       (date) datetime64[ns] 14kB 2013-01-01 2013-01-02 ... 2017-08-15
  * component  (component) object 8B 'sales'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  static_covariates store_nbr      family   city      s...
    hierarchy:          None

We retrieve all the values indicated above: the number of sales, the date of each sale in **Coordinates > date**, and the dependent covariates in **Attributes > static_covariates**.

You can also see that the length of the time series is 1688. Originally it was 1684 but we added the values of the four December 25s that are missing from the dataset.

Then we apply a normalization to our **TimeSeries**.

#### Normalizing time series
Normalization is a technique used to improve the performance of a Machine Learning model by facilitating its training. I let you refer to our article on the subject if you want to know more.

We can easily normalize a **TimeSeries** with the **Scaler** function of darts.

Moreover, we will further optimize the training of the model by one hot encoding our covariates. We implement the one hot encoding via the **StaticCovariatesTransformer** function.

In [ ]:
from darts.dataprocessing import Pipeline
from darts.dataprocessing.transformers import Scaler, StaticCovariatesTransformer, MissingValuesFiller, InvertibleMapper
import sklearn

family_pipeline_dict = {}
family_TS_transformed_dict = {}

for key in family_TS_dict:
  train_filler = MissingValuesFiller(verbose=False, n_jobs=-1, name="Fill NAs")
  static_cov_transformer = StaticCovariatesTransformer(verbose=False, transformer_cat = sklearn.preprocessing.OneHotEncoder(), name="Encoder")
  log_transformer = InvertibleMapper(np.log1p, np.expm1, verbose=False, n_jobs=-1, name="Log-Transform")
  train_scaler = Scaler(verbose=False, n_jobs=-1, name="Scaling")

  train_pipeline = Pipeline([train_filler,
                             static_cov_transformer,
                             log_transformer,
                             train_scaler])

  training_transformed = train_pipeline.fit_transform(family_TS_dict[key])
  family_pipeline_dict[key] = train_pipeline
  family_TS_transformed_dict[key] = training_transformed

We can display the first transformed **TimeSeries** of the first family:

In [ ]:
display(family_TS_transformed_dict['AUTOMOTIVE'][0])

<TimeSeries (DataArray) (date: 1688, component: 1, sample: 1)> Size: 14kB
array([[[0.        ]],

       [[0.36672579]],

       [[0.46275643]],

       ...,

       [[0.23137821]],

       [[0.23137821]],

       [[0.53724357]]])
Coordinates:
  * date       (date) datetime64[ns] 14kB 2013-01-01 2013-01-02 ... 2017-08-15
  * component  (component) object 8B 'sales'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  static_covariates  store_nbr_1  store_nbr_10  store_n...
    hierarchy:          None

You can see that the sales have been normalized and that the **static_covariates** have been one hot encoded.

We now have our main time series that will allow us to train our model.

#### Covariates
A covariate is a variable that helps to predict a target variable.

This covariate can be dependent on the target variable. For example, the type of store, **type**, where the sales are made. But it can also be independent. For example, the price of oil on the day of the sale of a product.

This covariate can be known in advance, for example in our dataset we have the price of oil from January 1, 2013 to August 31, 2017. In this case, we talk about a **future covariate**.

There are also **past covariates**. These are covariates that are not known in advance. For example in our dataset, the transactions are known for the dates January 1, 2013 to August 15, 2017.

#### Date
The first covariate we are interested in is the date.

The date is a future covariate because we know the date of the coming days.

It has, in many cases, an impact on the traffic of a store. For example, we can expect that on Saturday there will be more customers in the store than on Monday.

But it can also be expected that during the summer vacations the store will be less busy than in normal times.

** Hence every little detail counts**.

In order not to miss anything, we will extract as much information as possible from this date. Here, 7 columns :

* year – year
* month – month
* day – day
* dayofyear – day of the year (for example February 1 is the 32nd day of the year)
* weekday – day of the week (there are 7 days in a week)
* weekofyear – week of the year (there are 52 weeks in a year)
* linear_increase – the index of the interval

In [ ]:
from darts.utils.timeseries_generation import datetime_attribute_timeseries

full_time_period = pd.date_range(start='2013-01-01', end='2017-08-31', freq='D')


year = datetime_attribute_timeseries(time_index = full_time_period, attribute="year")
month = datetime_attribute_timeseries(time_index = full_time_period, attribute="month")
day = datetime_attribute_timeseries(time_index = full_time_period, attribute="day")
dayofyear = datetime_attribute_timeseries(time_index = full_time_period, attribute="dayofyear")
weekday = datetime_attribute_timeseries(time_index = full_time_period, attribute="dayofweek")
weekofyear = datetime_attribute_timeseries(time_index = full_time_period, attribute="weekofyear")
timesteps = TimeSeries.from_times_and_values(times=full_time_period,
                                             values=np.arange(len(full_time_period)),
                                             columns=["linear_increase"])

time_cov = year.stack(month).stack(day).stack(dayofyear).stack(weekday).stack(weekofyear).stack(timesteps)
time_cov = time_cov.astype(np.float32)

This is what it gives us for the date at index 100:

In [ ]:
display(print(time_cov.components.values))
display(time_cov[100])

['year' 'month' 'day' 'dayofyear' 'dayofweek' 'weekofyear'
 'linear_increase']


None

/usr/local/lib/python3.10/dist-packages/darts/timeseries.py:4751: FutureWarning: DatetimeIndex.is_integer is deprecated. Use pandas.api.types.is_integer_dtype instead.
  if time_idx.is_integer() and not isinstance(time_idx, pd.RangeIndex):


<TimeSeries (DataArray) (time: 1, component: 7, sample: 1)> Size: 28B
array([[[2013.],
        [   4.],
        [  11.],
        [ 101.],
        [   3.],
        [  15.],
        [ 100.]]], dtype=float32)
Coordinates:
  * time       (time) datetime64[ns] 8B 2013-04-11
  * component  (component) object 56B 'year' 'month' ... 'linear_increase'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

And of course, we will normalize this data:


In [ ]:
time_cov_scaler = Scaler(verbose=False, n_jobs=-1, name="Scaler")
time_cov_train, time_cov_val = time_cov.split_before(pd.Timestamp('20170816'))
time_cov_scaler.fit(time_cov_train)
time_cov_transformed = time_cov_scaler.transform(time_cov)

You can also see that a split is made between the dates before August 15, 2017 and after (dates that will be used in the prediction).

#### Oil
As said before, the price of oil is a future covariate because it is known in advance.

Here, we will not simply extract the daily oil price but we will calculate the moving average.

The **moving average in X**, is an average of the current value and the X-1 previous values of a time series.

For example the moving average in 7 is the average of (t + t-1 + … + t-6) / 7. It is calculated at each t, that’s why it is called “moving”.

**Calculating the moving average allows us to remove the momentary fluctuations of a value and thus to accentuate the long-term trends**.

The moving average is used in trading, but more generally in Time Series Analysis.

In the following code, we calculate the moving average in 7 and 28 of the oil price. And of course, we apply a normalization :

In [ ]:
!pip install --upgrade scikit-learn==1.2.2

In [ ]:
from darts.models import MovingAverage

# Oil Price

oil = TimeSeries.from_dataframe(data_oil,
                                time_col = 'date',
                                value_cols = ['dcoilwtico'],
                                freq = 'D')

oil = oil.astype(np.float32)

# Transform
oil_filler = MissingValuesFiller(verbose=False, n_jobs=-1, name="Filler")
oil_scaler = Scaler(verbose=False, n_jobs=-1, name="Scaler")
oil_pipeline = Pipeline([oil_filler, oil_scaler])
oil_transformed = oil_pipeline.fit_transform(oil)

# Moving Averages for Oil Price
oil_moving_average_7 = MovingAverage(window=7)
oil_moving_average_28 = MovingAverage(window=28)

oil_moving_averages = []

ma_7 = oil_moving_average_7.filter(oil_transformed).astype(np.float32)
ma_7 = ma_7.with_columns_renamed(col_names=ma_7.components, col_names_new="oil_ma_7")
ma_28 = oil_moving_average_28.filter(oil_transformed).astype(np.float32)
ma_28 = ma_28.with_columns_renamed(col_names=ma_28.components, col_names_new="oil_ma_28")
oil_moving_averages = ma_7.stack(ma_28)

/usr/local/lib/python3.10/dist-packages/dask/dataframe/__init__.py:42: FutureWarning: 
Dask dataframe query planning is disabled because dask-expr is not installed.

You can install it with `pip install dask[dataframe]` or `conda install dask`.
This will raise in a future version.

  warnings.warn(msg, FutureWarning)


Here is the result obtained at index 100:

In [ ]:
display(oil_moving_averages[100])

/usr/local/lib/python3.10/dist-packages/darts/timeseries.py:4751: FutureWarning: DatetimeIndex.is_integer is deprecated. Use pandas.api.types.is_integer_dtype instead.
  if time_idx.is_integer() and not isinstance(time_idx, pd.RangeIndex):


<TimeSeries (DataArray) (date: 1, component: 2, sample: 1)> Size: 8B
array([[[0.78416604],
        [0.78240633]]], dtype=float32)
Coordinates:
  * date       (date) datetime64[ns] 8B 2013-04-11
  * component  (component) object 16B 'oil_ma_7' 'oil_ma_28'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

#### Holidays
Here, Ferdinand Berr has implemented functions to detail these holidays. In particular, he adds information about whether the holiday is Christmas day, whether it is a soccer game day, etc:

In [ ]:
def holiday_list(data_stores):

    listofseries = []

    for i in range(0,len(data_stores)):

            df_holiday_dummies = pd.DataFrame(columns=['date'])
            df_holiday_dummies["date"] = data_holidays["date"]

            df_holiday_dummies["national_holiday"] = np.where(((data_holidays["type"] == "Holiday") & (data_holidays["locale"] == "National")), 1, 0)

            df_holiday_dummies["earthquake_relief"] = np.where(data_holidays['description'].str.contains('Terremoto Manabi'), 1, 0)

            df_holiday_dummies["christmas"] = np.where(data_holidays['description'].str.contains('Navidad'), 1, 0)

            df_holiday_dummies["football_event"] = np.where(data_holidays['description'].str.contains('futbol'), 1, 0)

            df_holiday_dummies["national_event"] = np.where(((data_holidays["type"] == "Event") & (data_holidays["locale"] == "National") & (~data_holidays['description'].str.contains('Terremoto Manabi')) & (~data_holidays['description'].str.contains('futbol'))), 1, 0)

            df_holiday_dummies["work_day"] = np.where((data_holidays["type"] == "Work Day"), 1, 0)

            df_holiday_dummies["local_holiday"] = np.where(((data_holidays["type"] == "Holiday") & ((data_holidays["locale_name"] == data_stores['state'][i]) | (data_holidays["locale_name"] == data_stores['city'][i]))), 1, 0)

            listofseries.append(df_holiday_dummies)

    return listofseries

Then, we have a function to remove the days equal to 0 and the duplicates:


In [ ]:
def remove_0_and_duplicates(holiday_list):

    listofseries = []

    for i in range(0,len(holiday_list)):

            df_holiday_per_store = list_of_holidays_per_store[i].set_index('date')

            df_holiday_per_store = df_holiday_per_store.loc[~(df_holiday_per_store==0).all(axis=1)]

            df_holiday_per_store = df_holiday_per_store.groupby('date').agg({'national_holiday':'max', 'earthquake_relief':'max',
                                   'christmas':'max', 'football_event':'max',
                                   'national_event':'max', 'work_day':'max',
                                   'local_holiday':'max'}).reset_index()

            listofseries.append(df_holiday_per_store)

    return listofseries


And finally a function that allows us to have the holidays associated to each of the 54 stores :



In [ ]:
def holiday_TS_list_54(holiday_list):

    listofseries = []

    for i in range(0,54):

            holidays_TS = TimeSeries.from_dataframe(list_of_holidays_per_store[i],
                                        time_col = 'date',
                                        fill_missing_dates=True,
                                        fillna_value=0,
                                        freq='D')

            holidays_TS = holidays_TS.slice(pd.Timestamp('20130101'),pd.Timestamp('20170831'))
            holidays_TS = holidays_TS.astype(np.float32)
            listofseries.append(holidays_TS)

    return listofseries

In [ ]:
list_of_holidays_per_store = holiday_list(data_stores)
list_of_holidays_per_store = remove_0_and_duplicates(list_of_holidays_per_store)
list_of_holidays_store = holiday_TS_list_54(list_of_holidays_per_store)

holidays_filler = MissingValuesFiller(verbose=False, n_jobs=-1, name="Filler")
holidays_scaler = Scaler(verbose=False, n_jobs=-1, name="Scaler")

holidays_pipeline = Pipeline([holidays_filler, holidays_scaler])
holidays_transformed = holidays_pipeline.fit_transform(list_of_holidays_store)

#### We get 54 TimeSeries with 7 columns:

* national_holiday
* earthquake_relief
* christmas
* football_event
* national_event
* work_day
* local_holiday

Here is the **TimeSeries** index **100** for the first store:


In [ ]:
display(len(holidays_transformed))
display(holidays_transformed[0].components.values)
display(holidays_transformed[0][100])

54

array(['national_holiday', 'earthquake_relief', 'christmas',
       'football_event', 'national_event', 'work_day', 'local_holiday'],
      dtype=object)

/usr/local/lib/python3.10/dist-packages/darts/timeseries.py:4751: FutureWarning: DatetimeIndex.is_integer is deprecated. Use pandas.api.types.is_integer_dtype instead.
  if time_idx.is_integer() and not isinstance(time_idx, pd.RangeIndex):


<TimeSeries (DataArray) (date: 1, component: 7, sample: 1)> Size: 28B
array([[[0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.],
        [0.]]], dtype=float32)
Coordinates:
  * date       (date) datetime64[ns] 8B 2013-04-11
  * component  (component) object 56B 'national_holiday' ... 'local_holiday'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

#### Promotion
The last future covariate to process is the **onpromotion** column.

It gives us the number of items on promotion in a product family.

Here the code is similar to the one used for the **sales** column. It allows to extract for each family, the time series of the 54 stores:

In [ ]:
df_promotion = pd.concat([data_train, data_test], axis=0)
df_promotion = df_promotion.sort_values(["store_nbr","family","date"])
df_promotion.tail()

family_promotion_dict = {}

for family in family_list:
  df_family = df_promotion.loc[df_promotion['family'] == family]

  list_of_TS_promo = TimeSeries.from_group_dataframe(
                                df_family,
                                time_col="date",
                                group_cols=["store_nbr","family"],
                                value_cols="onpromotion",
                                fill_missing_dates=True,
                                freq='D')

  for ts in list_of_TS_promo:
    ts = ts.astype(np.float32)

  family_promotion_dict[family] = list_of_TS_promo

We can display the first **TimeSeries** of the first family :

In [ ]:
display(family_promotion_dict['AUTOMOTIVE'][0])

<TimeSeries (DataArray) (date: 1704, component: 1, sample: 1)> Size: 14kB
array([[[0.]],

       [[0.]],

       [[0.]],

       ...,

       [[0.]],

       [[0.]],

       [[0.]]])
Coordinates:
  * date       (date) datetime64[ns] 14kB 2013-01-01 2013-01-02 ... 2017-08-31
  * component  (component) object 8B 'onpromotion'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  static_covariates  store_nbr      family\ncomponent  ...
    hierarchy:          None

Let’s go further by calculating also the moving average in 7 and 28, like for the oil price:

In [ ]:
from tqdm import tqdm

promotion_transformed_dict = {}

for key in tqdm(family_promotion_dict):
  promo_filler = MissingValuesFiller(verbose=False, n_jobs=-1, name="Fill NAs")
  promo_scaler = Scaler(verbose=False, n_jobs=-1, name="Scaling")

  promo_pipeline = Pipeline([promo_filler,
                             promo_scaler])

  promotion_transformed = promo_pipeline.fit_transform(family_promotion_dict[key])

  # Moving Averages for Promotion Family Dictionaries
  promo_moving_average_7 = MovingAverage(window=7)
  promo_moving_average_28 = MovingAverage(window=28)

  promotion_covs = []

  for ts in promotion_transformed:
    ma_7 = promo_moving_average_7.filter(ts)
    ma_7 = TimeSeries.from_series(ma_7.pd_series())
    ma_7 = ma_7.astype(np.float32)
    ma_7 = ma_7.with_columns_renamed(col_names=ma_7.components, col_names_new="promotion_ma_7")
    ma_28 = promo_moving_average_28.filter(ts)
    ma_28 = TimeSeries.from_series(ma_28.pd_series())
    ma_28 = ma_28.astype(np.float32)
    ma_28 = ma_28.with_columns_renamed(col_names=ma_28.components, col_names_new="promotion_ma_28")
    promo_and_mas = ts.stack(ma_7).stack(ma_28)
    promotion_covs.append(promo_and_mas)

  promotion_transformed_dict[key] = promotion_covs

100%|██████████| 33/33 [01:24<00:00,  2.56s/it]


We obtain a normalized time series with 3 columns.

We can display the index 1 of the first TimeSeries of the first family:

In [ ]:
display(promotion_transformed_dict['AUTOMOTIVE'][0].components.values)
display(promotion_transformed_dict['AUTOMOTIVE'][0][1])

array(['onpromotion', 'promotion_ma_7', 'promotion_ma_28'], dtype=object)

/usr/local/lib/python3.10/dist-packages/darts/timeseries.py:4751: FutureWarning: DatetimeIndex.is_integer is deprecated. Use pandas.api.types.is_integer_dtype instead.
  if time_idx.is_integer() and not isinstance(time_idx, pd.RangeIndex):


<TimeSeries (DataArray) (date: 1, component: 3, sample: 1)> Size: 24B
array([[[0.],
        [0.],
        [0.]]])
Coordinates:
  * date       (date) datetime64[ns] 8B 2013-01-02
  * component  (component) object 24B 'onpromotion' ... 'promotion_ma_28'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  static_covariates  store_nbr      family\nglobal_comp...
    hierarchy:          None

#### Grouping the covariates
To finish with the future covariates, we are going to gather them in the same **TimeSeries**.

We start with the time series of the dates, the oil price and the moving averages of the oil price that we group in the variable **general_covariates**:

In [ ]:
general_covariates = time_cov_transformed.stack(oil_transformed).stack(oil_moving_averages)

Then for each store, we gather the **TimeSeries** of the holidays with the **general_covariates**:

In [ ]:
store_covariates_future = []

for store in range(0,len(store_list)):
  stacked_covariates = holidays_transformed[store].stack(general_covariates)
  store_covariates_future.append(stacked_covariates)

Finally, for each family, we combine the previously created covariates with the promotion covariates:

In [ ]:
future_covariates_dict = {}

for key in tqdm(promotion_transformed_dict):

  promotion_family = promotion_transformed_dict[key]
  covariates_future = [promotion_family[i].stack(store_covariates_future[i]) for i in range(0,len(promotion_family))]

  future_covariates_dict[key] = covariates_future

100%|██████████| 33/33 [00:06<00:00,  4.78it/s]


Here are the different columns obtained for each **TimeSeries** of each family of each store:

In [ ]:
display(future_covariates_dict['AUTOMOTIVE'][0].components)

Index(['onpromotion', 'promotion_ma_7', 'promotion_ma_28', 'national_holiday',
       'earthquake_relief', 'christmas', 'football_event', 'national_event',
       'work_day', 'local_holiday', 'year', 'month', 'day', 'dayofyear',
       'dayofweek', 'weekofyear', 'linear_increase', 'dcoilwtico', 'oil_ma_7',
       'oil_ma_28'],
      dtype='object', name='component')

### Transactions – Past Covariates
Before launching the training of the model, let’s extract the past covariates: the transactions.

As you might already have understand, after having taken the transactions for each store, we will normalize them:

In [ ]:
data_transactions.sort_values(["store_nbr","date"], inplace=True)

TS_transactions_list = TimeSeries.from_group_dataframe(
                                data_transactions,
                                time_col="date",
                                group_cols=["store_nbr"],
                                value_cols="transactions",
                                fill_missing_dates=True,
                                freq='D')

transactions_list = []

for ts in TS_transactions_list:
            series = TimeSeries.from_series(ts.pd_series())
            series = series.astype(np.float32)
            transactions_list.append(series)

transactions_list[24] = transactions_list[24].slice(start_ts=pd.Timestamp('20130102'), end_ts=pd.Timestamp('20170815'))

from datetime import datetime, timedelta

transactions_list_full = []

for ts in transactions_list:
  if ts.start_time() > pd.Timestamp('20130101'):
    end_time = (ts.start_time() - timedelta(days=1))
    delta = end_time - pd.Timestamp('20130101')
    zero_series = TimeSeries.from_times_and_values(
                              times=pd.date_range(start=pd.Timestamp('20130101'),
                              end=end_time, freq="D"),
                              values=np.zeros(delta.days+1))
    ts = zero_series.append(ts)
    ts = ts.with_columns_renamed(col_names=ts.components, col_names_new="transactions")
    transactions_list_full.append(ts)

transactions_filler = MissingValuesFiller(verbose=False, n_jobs=-1, name="Filler")
transactions_scaler = Scaler(verbose=False, n_jobs=-1, name="Scaler")

transactions_pipeline = Pipeline([transactions_filler, transactions_scaler])
transactions_transformed = transactions_pipeline.fit_transform(transactions_list_full)

Here is the **TimeSeries** for the first store:

In [ ]:
display(transactions_transformed[0])

<TimeSeries (DataArray) (time: 1688, component: 1, sample: 1)> Size: 14kB
array([[[0.        ]],

       [[0.69831293]],

       [[0.60635131]],

       ...,

       [[0.13761164]],

       [[0.57327158]],

       [[0.5600397 ]]])
Coordinates:
  * time       (time) datetime64[ns] 14kB 2013-01-01 2013-01-02 ... 2017-08-15
  * component  (component) object 8B 'transactions'
Dimensions without coordinates: sample
Attributes:
    static_covariates:  None
    hierarchy:          None

Finally ready to create our Machine Learning model.

## Machine Learning Model
Now, we will train a first Machine Learning model with the darts library to confirm that our data is consistent and that the predictions obtained are convincing.

Then we will use ensemble methods to improve our final result.

### Single model
The Darts library offers us various Machine Learning models to use on **TimeSeries**.

In Ferdinand Berr’s solution, we can see that he uses different models:

* **NHiTSModel** – score : 0.43265
* **RNNModel** (with LSTM layers) – score : 0.55443
* **TFTModel** – score : 0.43226
* **ExponentialSmoothing** – score : 0.37411
These scores are obtained on validation data, artificially generated from the training data.

Personally, I decided to use the LightGBMModel, an implementation of the eponymous library model on which you will find an article here.

Why use this model ? Not after hours of practice and experimentation, but simply by using it and seeing that, alone, it gives me better results than the **ExponentialSmoothing**.

As explained in the Strategy section, we will train a Machine Learning model for each product family.

So for each family, we have to take the corresponding **TimeSeries** and send them to our Machine Learning model.

First, we prepare the data:

* **TCN_covariates** represents the future covariates associated with the target product family
* **train_sliced** represents the number of sales associated with the target product family. The **slice_intersect** function that you can see used simply ensures that the components span the same time interval. In the case of different time intervals an error message will appear if we try to combine them.
* **transactions_transformed**, the past covariates do not need to be indexed on the target family because there is only one global **TimeSeries** per store
Next, we initialize hyperparameters for our model.

**This is the key to model results**.

By modifying these hyperparameters you can improve the performance of the Machine Learning model.

#### Training
Here are the important hyperparameters:

* **lags** – the number of past values on which we base our predictions
* **lags_future_covariates** – the number of future covariate values on which we base our predictions. If we give a tuple, the left value represents the number of covariates in the past and the right value represents the number of covariates in the future
* **lags_past_covariates** – the number of past covariate values on which we base our predictions
For these three hyperparameters, if a list is passed, we take the indexes associated with the numbers of this list. For example if we pass: [-3, -4, -5], we take the indexes t-3, t-4, t-5. But if we pass an integer for example 10, we take the 10 previous values (or the 10 future values depending on the case).

The hyperparameters **output_chunk_length** controls the number of predicted values in the future, **random_state** ensures the reproducibility of the results and **gpu_use_dp** indicates if we want to use a GPU.

After that we launch the training. And at the end, we save the trained model in a dictionary.

In [ ]:
from darts.models import LightGBMModel

LGBM_Models_Submission = {}

display("Training...")

for family in tqdm(family_list):

  sales_family = family_TS_transformed_dict[family]
  training_data = [ts for ts in sales_family]
  TCN_covariates = future_covariates_dict[family]
  train_sliced = [training_data[i].slice_intersect(TCN_covariates[i]) for i in range(0,len(training_data))]

  LGBM_Model_Submission = LightGBMModel(lags = 63,
                                        lags_future_covariates = (14,1),
                                        lags_past_covariates = [-16,-17,-18,-19,-20,-21,-22],
                                        output_chunk_length=1,
                                        random_state=2022,
                                        gpu_use_dp= "false",
                                        )

  LGBM_Model_Submission.fit(series=train_sliced,
                        future_covariates=TCN_covariates,
                        past_covariates=transactions_transformed)

  LGBM_Models_Submission[family] = LGBM_Model_Submission

'Training...'

  0%|          | 0/33 [00:00<?, ?it/s]

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.180601 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 43593
[LightGBM] [Info] Number of data points in the train set: 87750, number of used features: 484
[LightGBM] [Info] Start training from score 0.461126


  3%|▎         | 1/33 [00:19<10:25, 19.54s/it]

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.188509 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 28071
[LightGBM] [Info] Number of data points in the train set: 87750, number of used features: 484
[LightGBM] [Info] Start training from score 0.026578


...

 94%|█████████▍| 31/33 [10:21<00:43, 21.50s/it]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.746916 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 46005
[LightGBM] [Info] Number of data points in the train set: 87750, number of used features: 484
[LightGBM] [Info] Start training from score 0.087959


 97%|█████████▋| 32/33 [10:38<00:20, 20.15s/it]

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.239037 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 43458
[LightGBM] [Info] Number of data points in the train set: 87750, number of used features: 484
[LightGBM] [Info] Start training from score 0.512093


100%|██████████| 33/33 [10:58<00:00, 19.97s/it]


In the above code, we only use lags_past_covariates = [-16,-17,-18,-19,-20,-21,-22].

Because during the 16th prediction (the one of August 31, 2017), the values of the past covariates from -1 to -15 are not known.

After training, we obtain 33 Machine Learning models stored in LGBM_Models_Submission.

### Predict
We can now perform the predictions:

In [ ]:
display("Predictions...")

LGBM_Forecasts_Families_Submission = {}

for family in tqdm(family_list):

  sales_family = family_TS_transformed_dict[family]
  training_data = [ts for ts in sales_family]
  LGBM_covariates = future_covariates_dict[family]
  train_sliced = [training_data[i].slice_intersect(TCN_covariates[i]) for i in range(0,len(training_data))]

  forecast_LGBM = LGBM_Models_Submission[family].predict(n=16,
                                         series=train_sliced,
                                         future_covariates=LGBM_covariates,
                                         past_covariates=transactions_transformed)

  LGBM_Forecasts_Families_Submission[family] = forecast_LGBM

'Predictions...'

100%|██████████| 33/33 [00:35<00:00,  1.08s/it]


Note: even if the model has an **output_chunk_length** of 1, we can directly instruct it to predict 16 values in the future.

We now have our predictions. If you follow well, you know the next step.

Previously, we normalized our data with the **Scaler** function. So the predicted data are also normalized.

To de-normalize them we use the **inverse_transform** function on each **TimeSeries**:

In [ ]:
LGBM_Forecasts_Families_back_Submission = {}

for family in tqdm(family_list):

  LGBM_Forecasts_Families_back_Submission[family] = family_pipeline_dict[family].inverse_transform(LGBM_Forecasts_Families_Submission[family], partial=True)

100%|██████████| 33/33 [00:24<00:00,  1.33it/s]


Finally, here is the code that allows to go from the predicted time series cluster to the prediction DataFrame:

In [ ]:
for family in tqdm(LGBM_Forecasts_Families_back_Submission):
  for n in range(0,len(LGBM_Forecasts_Families_back_Submission[family])):
    if (family_TS_dict[family][n].univariate_values()[-21:] == 0).all():
        LGBM_Forecasts_Families_back_Submission[family][n] = LGBM_Forecasts_Families_back_Submission[family][n].map(lambda x: x * 0)

listofseries = []

for store in tqdm(range(0,54)):
  for family in family_list:
      oneforecast = LGBM_Forecasts_Families_back_Submission[family][store].pd_dataframe()
      oneforecast.columns = ['fcast']
      listofseries.append(oneforecast)

df_forecasts = pd.concat(listofseries)
df_forecasts.reset_index(drop=True, inplace=True)

# No Negative Forecasts
df_forecasts[df_forecasts < 0] = 0
forecasts_kaggle = pd.concat([df_test_sorted, df_forecasts.set_index(df_test_sorted.index)], axis=1)
forecasts_kaggle_sorted = forecasts_kaggle.sort_values(by=['id'])
forecasts_kaggle_sorted = forecasts_kaggle_sorted.drop(['date','store_nbr','family'], axis=1)
forecasts_kaggle_sorted = forecasts_kaggle_sorted.rename(columns={"fcast": "sales"})
forecasts_kaggle_sorted = forecasts_kaggle_sorted.reset_index(drop=True)

# Submission
submission_kaggle = forecasts_kaggle_sorted

100%|██████████| 54/54 [00:01<00:00, 42.53it/s]


Display the predictions:

In [ ]:
submission_kaggle.head()

,id,sales
0,3000888,3.137218
1,3000889,0.000000
2,3000890,3.962451
3,3000891,2276.543092
4,3000892,0.039745


Train several models and apply the Ensemble method.

### Multiple models
As explained before, the important thing in this code is the hyperparameters. We will train 3 models by taking the following hyperparameters:

In [ ]:
model_params = [
    {"lags" : 7, "lags_future_covariates" : (16,1), "lags_past_covariates" : [-16,-17,-18,-19,-20,-21,-22]},
    {"lags" : 365, "lags_future_covariates" : (14,1), "lags_past_covariates" : [-16,-17,-18,-19,-20,-21,-22]},
    {"lags" : 730, "lags_future_covariates" : (14,1), "lags_past_covariates" : [-16,-17,-18,-19,-20,-21,-22]}
]

For each of these parameters, we will train 33 models, run the predictions and fill the final DataFrame. The 3 DataFrames obtained will be stored in the **submission_kaggle_list**:

In [ ]:
from sklearn.metrics import mean_squared_log_error as msle, mean_squared_error as mse
from lightgbm import early_stopping

submission_kaggle_list = []

for params in model_params:

  LGBM_Models_Submission = {}

  display("Training...")

  for family in tqdm(family_list):

    # Define Data for family
    sales_family = family_TS_transformed_dict[family]
    training_data = [ts for ts in sales_family]
    TCN_covariates = future_covariates_dict[family]
    train_sliced = [training_data[i].slice_intersect(TCN_covariates[i]) for i in range(0,len(training_data))]

    LGBM_Model_Submission = LightGBMModel(lags = params["lags"],
                                          lags_future_covariates = params["lags_future_covariates"],
                                          lags_past_covariates = params["lags_past_covariates"],
                                          output_chunk_length=1,
                                          random_state=2022,
                                          gpu_use_dp= "false")

    LGBM_Model_Submission.fit(series=train_sliced,
                          future_covariates=TCN_covariates,
                          past_covariates=transactions_transformed)

    LGBM_Models_Submission[family] = LGBM_Model_Submission

  display("Predictions...")


  LGBM_Forecasts_Families_Submission = {}

  for family in tqdm(family_list):

    sales_family = family_TS_transformed_dict[family]
    training_data = [ts for ts in sales_family]
    LGBM_covariates = future_covariates_dict[family]
    train_sliced = [training_data[i].slice_intersect(TCN_covariates[i]) for i in range(0,len(training_data))]

    forecast_LGBM = LGBM_Models_Submission[family].predict(n=16,
                                          series=train_sliced,
                                          future_covariates=LGBM_covariates,
                                          past_covariates=transactions_transformed)

    LGBM_Forecasts_Families_Submission[family] = forecast_LGBM

  # Transform Back

  LGBM_Forecasts_Families_back_Submission = {}

  for family in tqdm(family_list):

    LGBM_Forecasts_Families_back_Submission[family] = family_pipeline_dict[family].inverse_transform(LGBM_Forecasts_Families_Submission[family], partial=True)

  # Prepare Submission in Correct Format

  for family in tqdm(LGBM_Forecasts_Families_back_Submission):
    for n in range(0,len(LGBM_Forecasts_Families_back_Submission[family])):
      if (family_TS_dict[family][n].univariate_values()[-21:] == 0).all():
          LGBM_Forecasts_Families_back_Submission[family][n] = LGBM_Forecasts_Families_back_Submission[family][n].map(lambda x: x * 0)

  listofseries = []

  for store in tqdm(range(0,54)):
    for family in family_list:
        oneforecast = LGBM_Forecasts_Families_back_Submission[family][store].pd_dataframe()
        oneforecast.columns = ['fcast']
        listofseries.append(oneforecast)

  df_forecasts = pd.concat(listofseries)
  df_forecasts.reset_index(drop=True, inplace=True)

  # No Negative Forecasts
  df_forecasts[df_forecasts < 0] = 0
  forecasts_kaggle = pd.concat([df_test_sorted, df_forecasts.set_index(df_test_sorted.index)], axis=1)
  forecasts_kaggle_sorted = forecasts_kaggle.sort_values(by=['id'])
  forecasts_kaggle_sorted = forecasts_kaggle_sorted.drop(['date','store_nbr','family'], axis=1)
  forecasts_kaggle_sorted = forecasts_kaggle_sorted.rename(columns={"fcast": "sales"})
  forecasts_kaggle_sorted = forecasts_kaggle_sorted.reset_index(drop=True)

  # Submission
  submission_kaggle_list.append(forecasts_kaggle_sorted)

'Training...'

  0%|          | 0/33 [00:00<?, ?it/s]

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.150873 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 32715
[LightGBM] [Info] Number of data points in the train set: 89964, number of used features: 468
[LightGBM] [Info] Start training from score 0.458941


  3%|▎         | 1/33 [00:16<08:38, 16.21s/it]

[LightGBM] [Info] Auto-choosing row-wise multi-threading, the overhead of testing was 0.193250 seconds.
You can set `force_row_wise=true` to remove the overhead.
And if memory is not enough, you can set `force_col_wise=true`.
[LightGBM] [Info] Total Bins 26651
[LightGBM] [Info] Number of data points in the train set: 89964, number of used features: 468
[LightGBM] [Info] Start training from score 0.025924


...

 97%|█████████▋| 32/33 [41:47<01:15, 75.51s/it]

[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 2.121599 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 213439
[LightGBM] [Info] Number of data points in the train set: 51732, number of used features: 1136
[LightGBM] [Info] Start training from score 0.549995


100%|██████████| 33/33 [43:12<00:00, 78.57s/it]


'Predictions...'

100%|██████████| 54/54 [00:01<00:00, 53.78it/s]


We end up with four prediction DataFrames that we will sum and average (this is the so-called ensemble method):

In [ ]:
data_sample_submission['sales'] = (submission_kaggle[['sales']]+submission_kaggle_list[0][['sales']]+submission_kaggle_list[1][['sales']]+submission_kaggle_list[2][['sales']])/4

Here is the result:

In [ ]:
data_sample_submission.head()

,id,sales
0,3000888,3.391998
1,3000889,0.000000
2,3000890,4.150837
3,3000891,2255.160117
4,3000892,0.025445


Save the predictions in a CSV file and submit it to Kaggle:

In [ ]:
data_sample_submission.to_csv('/kaggle/working/submission.csv', index=False)

OSError: Cannot save file into a non-existent directory: '/kaggle/working'